In [ ]:
import sys,os
sys.path.append("..")
import django
django.setup()
import pandas as pd
from io import StringIO
import requests
import datetime
from crawlers.finlab.pioneers import *
from crawlers.finlab.import_tools import *

In [ ]:
class CrawlStockTdccTW:
    def __init__(self):
        self.target_name = "台股集保餘額資訊"
        self.sub_market = ["sii", "otc"]

    @classmethod
    def crawl_main(cls, file=False):
        if file is False:
            headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_1\
            0_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}
            res = requests.get("https://smart.tdcc.com.tw/opendata/getOD.ashx?id=1-5", headers=headers)
            df = pd.read_csv(StringIO(res.text))
        else:
            df = pd.read_csv(file)
        df = df.astype(str)
        df = df[~df['證券代號'].str.contains('YY|YE')]
        df = df.rename(columns={
            '證券代號': 'stock_id', '持股分級': 'hold_class',
            '人數': 'people', '股數': 'hold_num', '占集保庫存數比例%': 'hold_pt'
        })
        if 'hold_pt' not in df.columns:
            df = df.rename(columns={'佔集保庫存數比例%': 'hold_pt'})
        df = df[df['hold_class'] != '16']
        df.iloc[:, 2:6] = df.iloc[:, 2:6].apply(lambda s: pd.to_numeric(s, errors="coerce"))
        df['date'] = df[df.columns[0]].apply(lambda s: datetime.datetime.strptime(s, '%Y%m%d'))
        df = df.drop(columns=df.columns[0])
        return df

    
# df=CrawlStockTdccTW.crawl_main()
# df

# import old csv files

In [ ]:
from os import listdir
from os.path import isdir,isfile, join

mypath='/Users/benbilly3/Downloads/CSV'
onlydirs = [f for f in listdir(mypath) if isdir(join(mypath, f))]
path_list=[mypath+'/'+p for p in onlydirs][:-1]
onlyfiles = [d+'/'+f for d in path_list for f in listdir(d) if not isfile(join(mypath, f))]
onlyfiles=onlyfiles[onlyfiles.index('/Users/benbilly3/Downloads/CSV/2016/1104.csv'):]
onlyfiles


In [ ]:
[AddToSQL.add_to_sql(StockTdccTW, CrawlStockTdccTW.crawl_main(file),pk_columns=['stock_id','date','hold_class']) for file in onlyfiles]


#  miss File

In [ ]:
# 20190920
# 20171229

# 單隻抓取

In [ ]:
def crawl_tdcc(date,stock_id):
   
    datestr = date.strftime('%Y%m%d')
   
    headers = {
    'User-Agent':'Mozilla/5.0 (Windows NT 6.1; WOW64; rv:2.0.1) Gecko/2010010' \
    '1 Firefox/4.0.1',
    'Accept':'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
    'Accept-Language':'en-us,en;q=0.5',
    'Accept-Charset':'ISO-8859-1,utf-8;q=0.7,*;q=0.7'}
   
    url = 'https://www.tdcc.com.tw/smWeb/QryStockAjax.do'
   
    payload={
    'scaDates':datestr,
    'scaDate':datestr,
    'SqlMethod':'StockNo',
    'StockNo':stock_id,
    'radioStockNo':stock_id,
    'StockName':'',
    'REQ_OPR':'SELECT',
    'clkStockNo':stock_id,
    'clkStockName':''}

    try:
        res = requests.post(url,data=payload,headers = headers,timeout=30)
    except:
        return None
    
   
    df = pd.read_html(StringIO(res.text))
    df=pd.DataFrame(df[0])
    df.columns=df.iloc[9]
    df=df.dropna(thresh=4)
    if len(df)<5:
        return None
    df=df.iloc[1:]
    df=df.drop(columns=['持股/單位數分級'])
    df['序']=df['序'].fillna('17')
    df=df.apply(lambda s: pd.to_numeric(s, errors="coerce"))
    df = df.rename(
        columns={k: v for k, v in zip(df.columns, ['hold_class', 'people', 'hold_num', 'hold_pt'])})
    df['stock_id']=stock_id
    print(stock_id)
    df['date']=date.date()
    df=df.dropna()

    return df

crawl_QryStock_stock(datetime.datetime(2019,9,20),'1101')

In [ ]:

x=list(CrawlStockPriceTW(datetime.datetime(2019,9,20)).crawl_main()['stock_id'].values)
x

In [ ]:
xx=pd.concat([crawl_tdcc(datetime.datetime(2019,9,20),s) for s in x])
xx.to_csv('/Users/benbilly3/Desktop/20190920tdcc.csv')

In [ ]:
pd.set_option("display.max_rows", None)
xx

In [ ]:
AddToSQL.add_to_sql(StockTdccTW, CrawlStockTdccTW.crawl_main('/Users/benbilly3/Desktop/20190920tdcc.csv'),pk_columns=['stock_id','date','hold_class'])